In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from scipy.sparse import coo_matrix
from sklearn.neighbors import NearestNeighbors
from surprise import SVD
import implicit

In [ ]:
df = pd.read_csv('/Users/aaronfalloon/Downloads/2ea54c57-868c-41f3-a397-18adac14ef15.csv')

In [ ]:
df.columns = ['service', 'contractor']

In [ ]:
df = df.query('contractor != "[null]"')

In [ ]:
services = []
contractors = []

In [ ]:
for row in df.itertuples(index=False):
    row_services = row[0][1: -1].split(',')
    row_contractors = row[1][1: -1].split(',')
    if len(row_services) > 1 and len(row_services) != len(row_contractors):
        continue
    if len(row_services) == 1:
        services += (row_services * len(row_contractors))
    else:
        services += row_services
    contractors += row_contractors

In [ ]:
final = pd.DataFrame()
final = final.assign(service=services)
final = final.assign(contractor=contractors)

In [ ]:
final.iloc[:10, :]

In [ ]:
services_unique = final['service'].unique()

In [ ]:
services_map = dict(zip(services_unique, range(0, len(services_unique))))

In [ ]:
contractors_unique = final['contractor'].unique()

In [ ]:
contractors_map = dict(zip(contractors_unique, range(0, len(contractors_unique))))

In [ ]:
service_by_contractor = np.zeros((len(services_unique), len(contractors_unique)))

In [ ]:
for row in final.itertuples(index=False):
    service_by_contractor[services_map[row[0]], contractors_map[row[1]]] = 1

In [ ]:
service_by_contractor_sparse = coo_matrix(service_by_contractor)

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [ ]:
model.fit(service_by_contractor_sparse)

In [ ]:
user_items = service_by_contractor_sparse.T.tocsr()

In [ ]:
user_id = 9500
print(contractors_unique[user_id])
recs = model.recommend(user_id, user_items)
indices = [rec[0] for rec in recs]
[(services_unique[rec[0]], rec[1]) for rec in recs]